In [1]:
# import modules
# these will bring in other python packages to help us analyze and visualize the data

import pandas as pd # data analysis package
import seaborn as sns # data visualization package
import json
pd.options.display.float_format = '{:.2}'.format

In [2]:
# set url variable to csv data in Github
url = "https://raw.githubusercontent.com/swatland610/data_viz_challenge/main/data/2021_chicago_budget_appropriations.csv"

# read data at url, bring it into a data table (DataFrame)
budget_appropriations = pd.read_csv(url)

In [4]:
budget_appropriations.sort_values('2021 ORDINANCE (AMOUNT $)').head()

,FUND TYPE,FUND CODE,FUND DESCRIPTION,DEPARTMENT NUMBER,DEPARTMENT DESCRIPTION,APPROPRIATION AUTHORITY,APPROPRIATION AUTHORITY DESCRIPTION,APPROPRIATION ACCOUNT,APPROPRIATION ACCOUNT DESCRIPTION,2021 ORDINANCE (AMOUNT $)
2076,LOCAL,100,CORPORATE FUND,99,FINANCE GENERAL,2005,FINANCE GENERAL,9333,LESS CORPORATE FUND,-25000000
2046,LOCAL,100,CORPORATE FUND,99,FINANCE GENERAL,2005,FINANCE GENERAL,94,LAYOFFS / FURLOUGH DAYS SAVINGS,-1304868
3264,LOCAL,740,O HARE REVENUE FUND,99,FINANCE GENERAL,2005,FINANCE GENERAL,94,LAYOFFS / FURLOUGH DAYS SAVINGS,-199482
2302,LOCAL,200,WATER FUND,99,FINANCE GENERAL,2005,FINANCE GENERAL,94,LAYOFFS / FURLOUGH DAYS SAVINGS,-144680
2504,LOCAL,300,VEHICLE FUND,99,FINANCE GENERAL,2005,FINANCE GENERAL,94,LAYOFFS / FURLOUGH DAYS SAVINGS,-100673
...,...,...,...,...,...,...,...,...,...,...
3273,LOCAL,740,O HARE REVENUE FUND,99,FINANCE GENERAL,2005,FINANCE GENERAL,902,INTEREST ON BONDS,451481468
1673,LOCAL,100,CORPORATE FUND,59,CFD,2005,Chicago Fire Department,5,SALARIES AND WAGES - ON PAYROLL,489623343
3064,LOCAL,681,MUNICIPAL ANNUNITY AND BENEFIT,99,FINANCE GENERAL,2005,FINANCE GENERAL,976,EMPLOYEE ANNUITY AND BENEFIT,576000000
3068,LOCAL,683,POLICEMEN'S ANNUITY AND BENEFIT,99,FINANCE GENERAL,2005,FINANCE GENERAL,976,EMPLOYEE ANNUITY AND BENEFIT,786793000


In [137]:
budget_appropriations["2021 ORDINANCE (AMOUNT $)"].sum()

13522521000

In [11]:
df = budget_appropriations.sort_values(dept, ignore_index=True)


In [119]:
def nest_data(df, nest_order: list):
    final_json = {'name': 'departments', 'children':[]}
    nest_order.append('2021 ORDINANCE (AMOUNT $)')
    df = df[nest_order]
    df_ls = [df]
    for col in nest_order:
        if col != nest_order[-2]:
            df_ls = data_runner(df_ls, col)
        else:
            result = process_dfs(df_ls)
            return result

def data_runner(df_ls, column):
    new_df_ls = list()
    for d in df_ls:
        col_value_ls = d[column].unique()
        ls = [d.loc[d[column]==value] for value in col_value_ls]
        new_df_ls.extend(ls)
    
    return new_df_ls

def process_dfs(df_ls):
    final_ls = list()
    for df in df_ls:
        col_n = df.shape[1]
        parents = [df.iloc[0, i] for i in range(col_n-2)]
        child = df.iloc[:, -2:]
        child_sum = child.groupby('APPROPRIATION ACCOUNT DESCRIPTION', as_index=False).sum()
        children_ls = [{"name":t._1,"value":t._2} for t in child_sum.itertuples()]
        children_dict = {"name":parents[-1], "children":children_ls}
        children_dict = {"name":parents[-2], "children":[children_dict]}
        final_ls.append(children_dict)
    return final_ls



In [133]:
fg = nest_df['DEPARTMENT DESCRIPTION'] == 'FINANCE GENERAL'
nest_df[fg]
#nest_df['DEPARTMENT DESCRIPTION'].value_counts()

,DEPARTMENT DESCRIPTION,APPROPRIATION AUTHORITY DESCRIPTION,APPROPRIATION ACCOUNT DESCRIPTION,2021 ORDINANCE (AMOUNT $)
2530,FINANCE GENERAL,FINANCE GENERAL,UNEMPLOYMENT INSURANCE CLAIM,15063
2531,FINANCE GENERAL,FINANCE GENERAL,HMO PREMIUMS,1465918
2532,FINANCE GENERAL,FINANCE GENERAL,For the Costs of Claims and Administration for...,4612464
2533,FINANCE GENERAL,FINANCE GENERAL,CITY DEFERRED COMPENSATION,211
2534,FINANCE GENERAL,FINANCE GENERAL,REIMB - PENSION PAYMENTS,134954
...,...,...,...,...
2861,FINANCE GENERAL,FINANCE GENERAL,FOR ANNUAL PAYMENT TO CTA,3000000
2862,FINANCE GENERAL,FINANCE GENERAL,DENTAL PLAN PREMIUMS,252028
2863,FINANCE GENERAL,FINANCE GENERAL,MEDICAL CARE CLAIMS,99768
2864,FINANCE GENERAL,FINANCE GENERAL,UNEMPLOYMENT INSURANCE CLAIM,174278


In [120]:
nest_order = ['DEPARTMENT DESCRIPTION', 'APPROPRIATION AUTHORITY DESCRIPTION', 'APPROPRIATION ACCOUNT DESCRIPTION']
test_ls = nest_data(df, nest_order)

In [134]:
nest_df = df[nest_order]
depts = df[nest_order].iloc[:,0].unique()
final_ls = []
for dept in depts:
    dept_ls = []
    dept_bool = nest_df.iloc[:,0] == dept
    dept_df = nest_df[dept_bool]
    dept_auth = dept_df.iloc[:,1].unique()
    for auth in dept_auth:
        auth_bool = nest_df.iloc[:,1] == auth
        auth_df = dept_df[auth_bool]
        child_df = auth_df.iloc[:, -2:]
        child_sum = child_df.groupby('APPROPRIATION ACCOUNT DESCRIPTION', as_index=False).sum()
        child_ls = [{"name":t._1,"level": "Account Description","value":t._2} for t in child_sum.itertuples()]
        auth_dict = {'name':auth, 'level':'Appropriation Authority', 'children':child_ls}
        dept_ls.append(auth_dict)
    dept_dict = {'name':dept, 'level':'Department', 'children': dept_ls}
    final_ls.append(dept_dict)
final_dict = {'name': 'budget', 'level':'Overall', 'children':final_ls}

<ipython-input-134-533d34c0ff32>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  auth_df = dept_df[auth_bool]


In [135]:
import json

with open('nested.json', 'w') as f:
    json.dump(final_dict, f)

In [124]:
rn = test_df.rename(columns={'APPROPRIATION ACCOUNT DESCRIPTION':'name', '2021 ORDINANCE (AMOUNT $)':'size'})
rn.to_json(orient='records')[:100]

'[{"DEPARTMENT DESCRIPTION":"AIS","APPROPRIATION AUTHORITY DESCRIPTION":"BUREAU OF FACILITY MANAGEMEN'

In [11]:
dept_desc = budget_appropriations.groupby('DEPARTMENT DESCRIPTION').sum()['2021 ORDINANCE (AMOUNT $)'].sort_values()

In [40]:
dept_desc/dept_desc.sum()

DEPARTMENT DESCRIPTION
LAC               1.4e-05
POLICE BOARD      4.2e-05
ETHICS            6.4e-05
CCHR              0.00021
CTO                0.0003
DHR                0.0005
CACC              0.00052
MOPD              0.00057
DOAH              0.00057
DPS               0.00063
CLERK             0.00078
OIG               0.00081
MO                0.00084
COPA              0.00098
ELECTIONS          0.0014
DCASE              0.0019
BACP                0.002
COUNCIL            0.0021
DOB                0.0025
LAW                 0.003
DOF                0.0063
DPD                0.0072
CPL                0.0082
OBM                0.0094
OEMC               0.0099
PSA                  0.01
DOH                 0.014
DSS                 0.021
DWM                 0.024
AIS                 0.033
CDPH                0.038
CFD                 0.055
CDOT                0.057
DFSS                0.058
CDA                 0.084
CPD                  0.13
FINANCE GENERAL      0.42
Name: 2021 ORDI